# 1、使用FUNCTION_CALL模式

In [10]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=1)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)


# 提供提示词模板（以ChatPromptTemplate为例）
# prompt_template = ChatPromptTemplate.from_messages([
#     ("system","你是一个乐于助人的ai助手，根据用户的提问，必要时调用Search工具，使用互联网检索数据"),
#     ("system","{agent_scratchpad}"),
#     ("system","{chat_history}"),  #添加一个chat_history的变量，用于记录上下文的记忆
#     ("human","{input}"),
#     # ("placeholder","{agent_scratchpad}")
# ])

prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手，根据用户的提问，必要时调用Search工具，使用互联网检索数据"),
    ("system","{chat_history}"),
    ("human","{input}"),
    ("system","{agent_scratchpad}")  #重大bug
])

# 提供记忆的实例：ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", #此值需要与提示词模板中的记录记忆的变量同名
)

# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询2025.10.30 lol全球总决赛谁打谁"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...

Invoking: `Search` with `2025 League of Legends World Championship match schedule October 30`


[{'title': '2025 World Championship - Match Schedule - Liquipedia', 'url': 'https://liquipedia.net/leagueoflegends/World_Championship/2025/Match_Schedule', 'content': "| 2025-10-30 07:00:00 October 30, 2025 - 15:00 CST | Quarterfinals | G2Image 118: G2 EsportsImage 119: G2 Esports | vs (Bo5) | Image 120: Top EsportsTES | View match details |\n| 2025-10-31 07:00:00 October 31, 2025 - 15:00 CST | Quarterfinals | ALImage 121: Anyone's LegendImage 122: Anyone's Legend | vs (Bo5) | Image 123: T1T1 | View match details |\n| 2025-11-01 07:00:00 November 1, 2025 - 15:00 CST | Semifinals | TBD | vs (Bo5) | TBD | View match details |", 'score': 0.93762916}]在2025年10月30日的《英雄联盟》全球总决赛中，将进行四分之一决赛，比赛对阵为：

- G2 Esports vs. Top Esports

比赛将于15:00 CST（中国标准时间）开始，采用五局三胜制（Bo5）。

你可以查看更详细的赛程信息 [这里](https://liquipedia.net/leagueoflegends/World_Championship/2025/Match_Schedul

In [11]:
result = agent_executor.invoke({"input":"明天谁打谁呢？"})
# 为啥一直重复查询？
print(result)



> Entering new AgentExecutor chain...

Invoking: `Search` with `2025年10月31日 lol 全球总决赛 赛程 对阵`


[{'title': '英雄聯盟2025賽季世界大賽 - 維基百科', 'url': 'https://zh.wikipedia.org/wiki/%E8%8B%B1%E9%9B%84%E8%81%94%E7%9B%9F2025%E8%B5%9B%E5%AD%A3%E5%85%A8%E7%90%83%E6%80%BB%E5%86%B3%E8%B5%9B', 'content': '比赛时间：2025年10月28日至11月9日（UTC+8）\n 比赛场馆： 上海梅赛德斯-奔驰文化中心（四分之一决赛及半决赛）、 成都东安湖体育公园多功能体育馆（总决赛）\n 淘汰赛共分为四分之一决赛，半决赛与总决赛三个阶段\n 8支队伍根据抽签产生四分之一决赛对手；来自瑞士轮阶段同一分組晋级的队伍不能成为四分之一决赛对手；隊伍積分3-0晉級的隊伍將不會分在同一半區，即启用半区规避原则\n 每阶段比赛均为Bo5比赛，胜者晋级下一阶段，直至總决赛\n\n### 赛程\n\n[编辑] [...] | 场次 | 蓝色方 | 结果 | | 红色方 |\n ---  --- \n| 1 |  |  |  |  |\n| 2 |  |  |  |  |\n| 3 |  |  |  |  |\n| 4 |  |  |  |  |\n| 5 |  |  |  |  |\n\n### 总决赛\n\n[编辑]\n\n|  |\n\n| 英雄联盟2025赛季全球总决赛 |\n|  |\n| 总决赛——2025年11月9日 |\n| 距今还有79天 |\n| 以当地时间计算 |\n| 如发现倒数时间不准确，请点击此处刷新 |\n\n 比赛时间：2025年11月9日（UTC+8）\n 比赛场馆： 成都东安湖体育公园多功能体育馆\n FMVP：TBD\n\n| 队伍 | 简称 | 胜场数 |\n --- \n| TBD | TBD |  |\n| TBD | TBD |  |\n\n| 场次 | 蓝色方 | 结果 | | 红色方 |\n ---  --- \n| 1 |  |  |  |  |\n| 2 |  |  |  | 

# 2、使用ReAct模式

举例1：使用网络上现成的提示词模板结构


In [8]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 使用LangChain Hub中的官方ReAct提示模板
prompt_template = hub.pull("hwchase17/react-chat")

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询2025年9月9日北京的天气情况"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "Beijing weather September 9 2025"
```[{'title': 'Weather in Beijing in September 2025 - Tripvenue', 'url': 'https://tripvenue.com/weather/china/l1816670/beijing/september', 'content': '| Sunshine duration (h) | 10 | 9.3 | 11.3 | 11.2 | 10.8 | 11.3 | 11 | 2.3 | 0 | 5.4 | 9.6 | 11.2 | 11 | 11.1 | 11 | 10.4 | 9.2 | 10 | 2.6 | 9.3 | 10.4 | 10 | 8.3 | 1 | 4.7 | 6.5 | 7.2 | 8.6 | 9.6 | 10.1 |\n| Precipitation (mm) | 0 | 3 | 0 | 0 | 1.1 | 0 | 0.2 | 38.2 | 28.5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3.3 | 0 | 0 | 0 | 0 | 0 | 0 | 0.5 | 0 | 0 | 0 | 0 | 0 | 0 | [...] | Precipitation hours | 0 | 5 | 0 | 0 | 5 | 0 | 1 | 14 | 15 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 |\n| Wind speed (m/s) | 11.4 | 17.2 | 9.7 | 7.9 | 8.6 | 11.9 | 18 | 14.1 | 13.9 | 9.7 | 15.5 | 9.7 | 13.6 | 16.6 | 16.9 | 10.3 | 16.9 | 15.9 | 17.1 | 10.5 | 15.9 | 22.3 | 9.5 | 7.1 | 9.7 | 1

In [9]:
result = agent_executor.invoke({"input":"珠海的呢？"})

print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: Search  
Action Input: "2025年9月9日珠海的天气情况"  [{'title': '【2025珠海天氣】珠海氣溫指南，解鎖最佳旅遊季節、絕美景點與 ...', 'url': 'https://hk.trip.com/blog/zhuhai-weather-tour/', 'content': '中國內地/香港/澳門/台灣 | 5G eSIM｜日費計劃／流量套餐｜24小時計費制｜1至30日｜QR碼\n\n## 珠海天氣 | 七日天氣預報\n\n為您提供即將前往目的地未來一周內的天氣預報，幫助您更好地規劃日常活動與出行計劃。\n\n|  |  |  |  |  |  |  |\n ---  ---  --- \n| 9月7日 | 9月8日 | 9月9日 | 9月10日 | 9月11日 | 9月12日 | 9月13日 |\n| 今日 | 明日 | 星期二 | 星期三 | 星期四 | 星期五 | 星期六 |\n| 34℃/29℃ | 33℃/26℃ | 30℃/26℃ | 30℃/26℃ | 30℃/27℃ | 32℃/27℃ | 32℃/27℃ |\n| 天晴 | 驟雨 | 大雨 | 中雨 | 驟雨 | 多雲 | 多雲 |\n\n## 珠海天氣 | 景點推介\n\n### 珠海長隆海洋王國\n\n珠海長隆海洋王國位於珠海長隆國際海洋度假區，是全球較大型的海洋主題樂園之一。園區共分為八大主題區，集各類珍稀海洋動物展館、動感遊樂設施、3大劇場表演於一體，還可享受環球滋味美食和購物樂趣，精彩不容錯過。\n\n八大主題區 [...] 埃塞俄比亞\n埃及\n摩洛哥\n南非\n尼日利亞\n塞舌爾\n突尼斯\n肯尼亞\n加納\n津巴布韋\n坦桑尼亞\n毛里求斯\n阿爾及利亞\n烏干達\n博茨瓦納\n\n旅遊體驗\n\n探索自然\n人文歷史\n休閒度假\n家庭出遊\n戶外運動\n購物\n美食\n酒店\n自駕遊\n水上活動\n優惠\n飛行體驗\n綠色出行\n旅行貼士\n公眾假期\n旅遊優惠\n季節精選\n旅遊資訊\n新冠肺炎\n\n# 【2025珠海天氣】珠海氣溫指南，解鎖最佳旅遊季節、絕美

举例2：自定义提示词模板

In [12]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="claude-haiku-4-5",  #5美元
    temperature=0,
)
# 自定义提示词模板
template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
"""
prompt_template = PromptTemplate.from_template(
    template=template,
)

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天lol谁打谁"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 今天LOL比赛 谁打谁
[{'title': 'LOL世界賽S15日期、隊伍、賽制、賽程持續更新中！Peanut含淚 ...', 'url': 'https://hk.trip.com/guide/info/LOL%E4%B8%96%E7%95%8C%E8%B3%BD.html', 'content': '這場 10 月 14 日於北京舉行的單場 BO5，將是 T1 與 IG 的生死之戰。 兩隊將直接交鋒，勝者將作為最後一支隊伍晉級 16 強瑞士輪，敗者則將直接被淘汰，無緣後續賽事。\n\n## 10.14入圍賽 IG 1:3 不敵T1\n\nFaker所領銜的T1，依然保持了連續13年世界賽中對面LPL戰隊BO5全勝的記錄。值得一提的是，S15賽季是Faker與T1第二次衝擊世界賽三連冠的偉大成就。Faker在今日賽後採訪時表示： 今天能在這麼多粉絲面前贏得比賽勝利，是非常值得開心的事情。被問到這是世界大賽歷史上第一次入圍賽就是生死戰，他說，雖然是背水一戰、輸掉比賽就要回家，但或許先開始藉此戰瞭解世界賽、先暖手，也是不錯體驗，所以他個人是樂觀去面對這件事。\n\n## LOL S15 世界賽瑞士輪解析\n\n《英雄聯盟》世界賽 S15 核心賽段瑞士輪（Swiss Stage）將在 10 月 15 日至 10 月 25 日期間舉行，距離開戰只剩下不到兩週時間。這將是全球 16 支頂尖戰隊爭奪八強淘汰賽席位的最關鍵階段！\n\n### 瑞士輪賽事詳情一覽 [...] |  |  |  |  |\n ---  --- |\n| 隊伍 | 對手 | 所在半區 | 關鍵分析 |\n| AL（LPL） | T1 (LCK) | 上半區 | AL 將面對狀態有所起伏的衛冕冠軍 T1，被視為 LPL 的上上籤。 |\n| TES(LPL) | G2 (LEC) | 上半區 | TES 對陣 G2 佔有優勢，上半區兩場對決 LPL 均被看好晉級。 |\n| GEN(LCK) | HLE (LCK) | 下半區 | LCK 內戰提前上演，幾乎鎖定一個決賽名額。 |\n| C

In [13]:
result = agent_executor.invoke({"input":"今天10.30 明天谁打谁呢？"})

print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: LOL英雄联盟S15世界赛10月31日比赛对阵
[{'title': '《英雄联盟》S15四分之一决赛赛程出炉：AL对阵T1 - 网易', 'url': 'https://www.163.com/dy/article/KCQ4NKCE0526D8LR.html', 'content': '### 新闻\n\n### 体育\n\n### 娱乐\n\n### 财经\n\n### 汽车\n\n### 科技\n\n### 时尚\n\n### 手机 / 数码\n\n### 房产 / 家居\n\n### 教育\n\n# 《英雄联盟》S15四分之一决赛赛程出炉：AL对阵T1\n\n用微信扫码二维码\n\n分享至好友和朋友圈\n\n2025《英雄联盟》S15全球总决赛已经结束瑞士轮争夺，八强队伍全部产生。\n\n韩国LCK赛区四支队伍全部晋级，中国大陆LPL赛区只有AL、TES晋级。\n\n2025全球总决赛淘汰赛四分之一决赛对阵情况如下：\n\n10月28日（星期二）15:00 HLE vs GEN\n\n10月29日（星期三）15:00 KT vs CFO\n\n10月30日（星期四）15:00 G2 vs TES\n\n10月31日（星期五）15:00 AL vs T1\n\n其中，HLE与GEN的比赛为韩国内战;CFO则是时隔十年，港澳台队伍再次晋级全球总决赛淘汰赛，他们将面对韩国LCK赛区三号种子KT战队。\n\nCFO战队是由中国台湾省选手组成的全华班战队，其队员均为中国籍选手。 [...] ### 荒唐！韩国刚对中国免签，就有人脱团！澳洲20人团全逃，20万押金都不要了！\n\n翁帆被曝已经搬出清华别墅！杨振宁留给她的现金在1亿元左右\n\n### 翁帆被曝已经搬出清华别墅！杨振宁留给她的现金在1亿元左右\n\n护栏钢化玻璃咋装的是透明塑料板？29岁小伙闪落景观桥骨折15处，住院多日维权难\n\n### 护栏钢化玻璃咋装的是透明塑料板？29岁小伙闪落景观桥骨折15处，住院多日维权难\n\n演员江一燕官宣离婚\n\n### 演员江一燕官宣离婚\n\n女友出门需

好吧，是我太穷了，换了claude-haiku-4.5效果一下就变好了